In [1]:
import numpy as np

from tqdm.auto import tqdm

In [2]:
import pyEulerCurves as pyecc

if pyecc.__version__ != "0.5.post":
    raise ValueError("wrong version!!")

In [3]:
from pyEulerCurves.distance_utils import difference_ECP, difference_ECP_naive

In [4]:
# pip install memory_profiler
%load_ext memory_profiler

In [5]:
def Hopf_bifurcation(x, y, B):

    Hopf_bifurcation = np.zeros((x.shape[0], y.shape[0], 2))
    Hopf_bifurcation[:, :, 0] = B * x - y - x * (x**2 + y**2)
    Hopf_bifurcation[:, :, 1] = x + B * y - y * (x**2 + y**2)

    return Hopf_bifurcation


x_min, x_max = -2, 2
y_min, y_max = -2, 2

SIZE = 1001

x_points, y_points = SIZE, SIZE

x = np.linspace(x_min, x_max, x_points)
y = np.linspace(y_min, y_max, y_points)
X, Y = np.meshgrid(x, y)

all_examples_Hb = []
b_values = np.array([-1, -0.1, 0.1, 1])
names_Hb = []

# set the seed for reproducibility
rng = np.random.default_rng(seed=42)

for b in b_values:
    # without noisy
    Hb_clean = Hopf_bifurcation(X, Y, b)
    all_examples_Hb.append(Hb_clean)
    names_Hb.append(f"Hopf bifurcation b={b:.1f} (clean)")

    # with noisy (1/3)
    noise_x = rng.normal(loc=0.0, scale=np.abs(X) / 3, size=X.shape)
    noise_y = rng.normal(loc=0.0, scale=np.abs(Y) / 3, size=Y.shape)
    Hb_noisy = Hb_clean.copy()
    Hb_noisy[:, :, 0] += noise_x
    Hb_noisy[:, :, 1] += noise_y

    all_examples_Hb.append(Hb_noisy)
    names_Hb.append(f"Hopf bifurcation b={b:.1f} (noisy)")

In [6]:
len(all_examples_Hb)

8

In [7]:
all_examples_Hb[0].shape

(1001, 1001, 2)

In [8]:
# compute contributions (just the first 2)
ecp_matrix_Hb = []
trans = pyecc.ECC_from_bitmap(multifiltration=True, workers=1)

for i, ex in tqdm(enumerate(all_examples_Hb[:2])):
    ecc = trans.fit_transform(ex)
    ecp_matrix_Hb.append(trans.contributions_list)

0it [00:00, ?it/s]

(1005004, 2)
Parallel part done
Elapsed time: 6.3 seconds
Merged 1003 dicts
Elapsed time: 3.2 seconds
(1005004, 2)
Parallel part done
Elapsed time: 6.6 seconds
Merged 1003 dicts
Elapsed time: 3.6 seconds


In [9]:
# Save to file
for i, ecp in enumerate(ecp_matrix_Hb):
    with open("data/hopf/hopf_{}".format(i), "w") as f:
        for (x, y), val in ecp:
            f.write(f"{x} {y} {val}\n")

In [10]:
## we need to figure out what are the max and min filtration values, so we can correctly compute the distances

print(np.min([c.min(axis=(0, 1)) for c in all_examples_Hb], axis=0))
print(np.max([c.max(axis=(0, 1)) for c in all_examples_Hb], axis=0))

[-21.32454426 -21.1318429 ]
[21.52111718 21.45778087]


In [11]:
## lets do from -22 to +22
## numba wants them to be floats
dims = ((-22.0, 22.0), (-22.0, 22.0))

In [12]:
for c in ecp_matrix_Hb:
    print(len(c))

1228557
1616483


In [13]:
%%time 
## time profiling
assert SIZE <= 201, 'this will probably run out ot memory, use the discretization below'

d1 = difference_ECP(ecp_matrix_Hb[0], ecp_matrix_Hb[1], dims=dims, verbose=True)
print(d1)

CPU times: user 3 μs, sys: 1e+03 ns, total: 4 μs
Wall time: 4.77 μs


AssertionError: this will probably run out ot memory, use the discretization below

In [ ]:
%%memit 
## memory profiling
assert SIZE <= 201, 'this will probably run out ot memory, use the discretization below'

d1 = difference_ECP(ecp_matrix_Hb[0], ecp_matrix_Hb[1], dims=dims, verbose=True)

In [ ]:
## way slower but uses less memory
# %%time
# d2 = difference_ECP_naive(ecp_matrix_Hb[0], ecp_matrix_Hb[1], dims=dims)
# print(d2)

## check the two functions return the same value
# assert d1 == d2

## C++ code
computes the distance by discretizing the ECPs on a regular grid, this reduces the input size and therefore the memory requirements

you can run terminal commands in a cell using `!`

In [14]:
# compile the code, modify ecp_2d.cpp to change the limits and grid size
! g++ -std=c++11 -O2 -o ecp_dist ecp_2d.cpp

In [15]:
# run with time and memory profiling
# on linux, change the -l flag to -v
!/usr/bin/time -l ./ecp_dist data/hopf/hopf_0 data/hopf/hopf_1

Combined bounding box: xmin = -22, xmax = 22, ymin = -22, ymax = 22, x_step = 10000, y_step = 10000
L1 distance = 1399753.63
        4.28 real         3.61 user         0.31 sys
          2082553856  maximum resident set size
                   0  average shared memory size
                   0  average unshared data size
                   0  average unshared stack size
              127292  page reclaims
                   1  page faults
                   0  swaps
                   0  block input operations
                   0  block output operations
                   0  messages sent
                   0  messages received
                   0  signals received
                   0  voluntary context switches
                4365  involuntary context switches
         53972590750  instructions retired
         12334868197  cycles elapsed
          2042894464  peak memory footprint


## Discretize
The same result can be achieved by first discretizing the ECPs and then using the python function to compute the distance.

In [16]:
from pyEulerCurves.distance_utils import discretize_contributions

as in the Cpp code, we discretize the contributions on a 10000x1000 grid on ((-22, 22), (-22, 22))

In [17]:
%%time
discrete_ecp1 = discretize_contributions(ecp_matrix_Hb[0], dims=dims, resolution=(10000, 10000))
discrete_ecp2 = discretize_contributions(ecp_matrix_Hb[1], dims=dims, resolution=(10000, 10000))

CPU times: user 7.86 s, sys: 226 ms, total: 8.09 s
Wall time: 8.22 s


In [18]:
%%memit
discrete_ecp1 = discretize_contributions(ecp_matrix_Hb[0], dims=dims, resolution=(10000, 10000))
discrete_ecp2 = discretize_contributions(ecp_matrix_Hb[1], dims=dims, resolution=(10000, 10000))

peak memory: 2122.31 MiB, increment: 274.22 MiB


In [19]:
%%time
difference_ECP(
    discrete_ecp1,
    discrete_ecp2,
    dims=dims,
    verbose=True,
)

using optimized numba 2d
computing L1 distance in the range (-22.0, -22.0) - (22.0, 22.0)
creating ECP matrix of size (8910, 9093)
CPU times: user 3.42 s, sys: 234 ms, total: 3.66 s
Wall time: 3.76 s


1399753.6391239224

In [20]:
%%memit
difference_ECP(
    discretize_contributions(ecp_matrix_Hb[0], dims=dims, resolution=(10000, 10000)),
    discretize_contributions(ecp_matrix_Hb[1], dims=dims, resolution=(10000, 10000)),
    dims=dims,
    verbose=True,
)

using optimized numba 2d
computing L1 distance in the range (-22.0, -22.0) - (22.0, 22.0)
creating ECP matrix of size (8910, 9093)
peak memory: 3837.61 MiB, increment: 1726.61 MiB


## Distance matrix

In [ ]:
# ecp_difference_matrix_Hb = np.zeros((len(ecp_matrix_Hb), len(ecp_matrix_Hb)))

# # compute difference
# for i in tqdm(range(len(ecp_matrix_Hb))):
#     for j in tqdm(range(i + 1, len(ecp_matrix_Hb))):
#         ecp_difference_matrix_Hb[i, j] = difference_ECP_fast(
#             ecp_matrix_Hb[i], ecp_matrix_Hb[j], dims=dims
#         )
#         ecp_difference_matrix_Hb[j, i] = ecp_difference_matrix_Hb[i, j]